In [101]:
from __future__ import annotations

import pandas as pd
from pandas.api.types import is_numeric_dtype

import numpy as np

## 0 - Preparation
### 0.0 - Data import

In [102]:
DATA_FILE = "data/cleaned data.xlsx"
DESCR_FILE = "data/Table_fields_merged.xlsx"

In [103]:
df_raw = pd.read_excel(DATA_FILE, sheet_name=1)
df_raw.head()

,name,essround,edition,proddate,idno,cntry,nwspol,netusoft,netustm,ppltrst,...,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dweight,pspwght,pweight,anweight
0,ESS9e03_1,9,3.1,17.02.2021,3,IT,60,4,120,2,...,0,1,0,0,0,0,1.019588,0.924885,1.909064,1.765664
1,ESS9e03_1,9,3.1,17.02.2021,13,IT,8888,5,240,5,...,0,1,0,0,0,0,1.038898,0.954811,1.909064,1.822796
2,ESS9e03_1,9,3.1,17.02.2021,31,IT,30,5,180,4,...,0,0,0,0,0,0,1.050053,0.816516,1.909064,1.558781
3,ESS9e03_1,9,3.1,17.02.2021,58,IT,900,5,180,6,...,0,0,0,0,0,0,0.872215,0.986662,1.909064,1.883601
4,ESS9e03_1,9,3.1,17.02.2021,122,IT,60,5,15,2,...,0,0,0,0,0,0,1.019588,1.436108,1.909064,2.741621


In [104]:
df_raw.describe()

,essround,edition,idno,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,...,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dweight,pspwght,pweight,anweight
count,2745.0,2.745000e+03,2745.000000,2745.000000,2745.000000,2745.000000,2745.000000,2745.000000,2745.000000,2745.000000,...,2745.000000,2745.000000,2745.000000,2745.000000,2745.0,2745.000000,2745.000000,2745.000000,2745.000000,2745.000000
mean,9.0,3.100000e+00,34868.096903,459.951184,3.635337,2735.492168,5.050638,5.575956,4.688889,2.973406,...,0.075774,0.029872,0.007286,0.256102,0.0,0.103825,1.000033,1.000000,1.909064,1.909064
std,0.0,4.441701e-16,20391.645391,1569.666239,1.650821,3249.315166,5.199096,8.129967,5.906410,0.916137,...,0.264685,0.170267,0.085062,0.436558,0.0,0.305089,0.066344,0.175315,0.000000,0.334687
min,9.0,3.100000e+00,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.581986,0.455010,1.909064,0.868642
25%,9.0,3.100000e+00,17495.000000,30.000000,2.000000,120.000000,3.000000,3.000000,3.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.972131,0.891878,1.909064,1.702651
50%,9.0,3.100000e+00,34654.000000,60.000000,5.000000,240.000000,5.000000,5.000000,4.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.010092,0.983470,1.909064,1.877508
75%,9.0,3.100000e+00,52624.000000,150.000000,5.000000,6666.000000,7.000000,6.000000,6.000000,4.000000,...,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,1.038898,1.096375,1.909064,2.093050
max,9.0,3.100000e+00,69961.000000,8888.000000,8.000000,8888.000000,88.000000,88.000000,88.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.096735,1.678103,1.909064,3.203605


In [105]:
# Create a copy of original data to be modified
df = df_raw.copy()

In [106]:
df_descr = pd.read_excel(DESCR_FILE)
df_descr.set_index("Name", drop=True, append=False, inplace=True)
df_descr.head()

,Label,Type,Format,Valid,Invalid,Question
Name,,,,,,
cntry,Country,discrete,character-2,49519.0,0.0,5 Country
dweight,Design weight,continuous,numeric-4.2,49519.0,0.0,R17 Design weight
pspwght,Post-stratification weight including design we...,continuous,numeric-4.2,49519.0,0.0,R18 Post-stratification weight including desig...
pweight,Population size weight (must be combined with ...,continuous,numeric-8.2,49519.0,0.0,R19 Population size weight (must be combined w...
anweight,Analysis weight,continuous,numeric-4.2,49519.0,0.0,Analysis weight


### 0.1 Create helper function to get summary of columns

In [107]:
def get_summary_table(df: pd.DataFrame,
                      target_col: str,
                      corr_method: str="pearson",
                      descr_df: pd.DataFrame=None,
                      descr_cols: list[str]=["Label", "Type"]) -> pd.DataFrame:
    """Return a pandas dataframe where each row corresponds to a column of df
    and each column contains some properties of that column from df.

    Parameters
    ----------
    df : pd.DataFrame
        The pandas dataframe of shape (N, K) containing the data.
    target_col : str
        The label of the column that contains the variable of interest which
        will be studied as the dependent variable. 
    corr_method : {‘pearson’, ‘kendall’, ‘spearman’} or callable
        Method to be used to compute the correlation with target col. By default
        is 'pearson'.
    descr_df : pd.DataFrame, optional
        If dataframe is passed, it will be merged on summary table. The passed
        dataframe should have the variables codes as index. If None, nothing
        happens. By default is None.
    descr_cols : list[str], optional
        List of columns labels to be imported from descr_df. By default is
        'Label' and 'Type'.

    Returns
    -------
    pd.DataFrame
        Dataframe of shape (K, 9).
    """
    df_summary = pd.DataFrame({
    "IS_NUMERIC": df.apply(lambda x: int(is_numeric_dtype(x))),
    "N_UNIQUE" : df.apply(lambda x: x.nunique()),
    "NA_ABS" : df.apply(lambda x: x.isna().sum()),    ## count of NAs in that column
    "NA_REL": df.apply(lambda x: x.isna().mean()),
    "MIN": df.apply(lambda x: x.min() if is_numeric_dtype(x) else np.nan),
    "MAX": df.apply(lambda x: x.max() if is_numeric_dtype(x) else np.nan),
    "MEAN": df.apply(lambda x: x.mean() if is_numeric_dtype(x) else np.nan),
    "MEDIAN": df.apply(lambda x: x.median() if is_numeric_dtype(x) else np.nan),
    "CORR_TARGET": df.apply(lambda x: x.corr(df[target_col], method=corr_method) if is_numeric_dtype(x) else np.nan),
    "SHARE_MOST_FREQ": df.apply(lambda x: x.value_counts(normalize=True).max()),
    }).round(2)
    
    if descr_df is None:
        return df_summary
    
    df_summary = pd.merge(
        df_summary,
        df_descr[descr_cols],
        how="left",
        left_index=True,
        right_index=True
    )
    df_summary.columns = df_summary.columns.str.upper()
    return df_summary

In [108]:
df_summary = get_summary_table(df, target_col="happy")
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
name,0,1,0,0.0,NaN,NaN,NaN,NaN,NaN,1.00
essround,1,1,0,0.0,9.00,9.00,9.00,9.00,NaN,1.00
edition,1,1,0,0.0,3.10,3.10,3.10,3.10,0.00,1.00
proddate,0,1,0,0.0,NaN,NaN,NaN,NaN,NaN,1.00
idno,1,2745,0,0.0,3.00,69961.00,34868.10,34654.00,0.01,0.00
...,...,...,...,...,...,...,...,...,...,...
hswrk,1,2,0,0.0,0.00,1.00,0.10,0.00,0.01,0.90
dweight,1,29,0,0.0,0.58,1.10,1.00,1.01,-0.01,0.10
pspwght,1,445,0,0.0,0.46,1.68,1.00,0.98,-0.00,0.02
pweight,1,1,0,0.0,1.91,1.91,1.91,1.91,NaN,1.00


## 1 - Variables selection


### 1.1 - Drop variables with a single value
We first drop the columns attaining a single value, as they provide no information at all.

In [109]:
unique_val_cols = df_summary[df_summary["N_UNIQUE"] == 1].index
unique_val_cols

Index(['name', 'essround', 'edition', 'proddate', 'cntry', 'dscrna', 'cmsrv',
       'pweight'],
      dtype='object')

In [110]:
df.drop(columns=unique_val_cols, inplace=True)

We can update the summary df, we see that we have now 176 variables.

In [111]:
df_summary = get_summary_table(df, target_col="happy")
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
idno,1,2745,0,0.0,3.00,69961.00,34868.10,34654.00,0.01,0.00
nwspol,1,297,0,0.0,0.00,8888.00,459.95,60.00,0.03,0.21
netusoft,1,7,0,0.0,1.00,8.00,3.64,5.00,0.07,0.50
netustm,1,63,0,0.0,0.00,8888.00,2735.49,240.00,-0.04,0.36
ppltrst,1,13,0,0.0,0.00,88.00,5.05,5.00,0.18,0.18
...,...,...,...,...,...,...,...,...,...,...
rtrd,1,2,0,0.0,0.00,1.00,0.26,0.00,-0.07,0.74
hswrk,1,2,0,0.0,0.00,1.00,0.10,0.00,0.01,0.90
dweight,1,29,0,0.0,0.58,1.10,1.00,1.01,-0.01,0.10
pspwght,1,445,0,0.0,0.46,1.68,1.00,0.98,-0.00,0.02


### 1.2 - Drop variables concerning the conduction of the study
There are a few columns merely related to the conduction of the study, we can drop them.

In [112]:
df.drop(columns=["idno", "dweight", "pspwght", "anweight"], inplace=True)

There are now 172 variables in the dataset.

In [113]:
df_summary = get_summary_table(df, target_col="happy")
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
nwspol,1,297,0,0.0,0.0,8888.0,459.95,60.0,0.03,0.21
netusoft,1,7,0,0.0,1.0,8.0,3.64,5.0,0.07,0.50
netustm,1,63,0,0.0,0.0,8888.0,2735.49,240.0,-0.04,0.36
ppltrst,1,13,0,0.0,0.0,88.0,5.05,5.0,0.18,0.18
pplfair,1,13,0,0.0,0.0,88.0,5.58,5.0,0.17,0.19
...,...,...,...,...,...,...,...,...,...,...
uempla,1,2,0,0.0,0.0,1.0,0.08,0.0,-0.05,0.92
uempli,1,2,0,0.0,0.0,1.0,0.03,0.0,0.01,0.97
dsbld,1,2,0,0.0,0.0,1.0,0.01,0.0,-0.01,0.99
rtrd,1,2,0,0.0,0.0,1.0,0.26,0.0,-0.07,0.74


### 1.3 - Missing values encoding
Missing values are not encoded as such in the dataset, but are instead replaced
by numeric values out of the ordinary scale. For the correctness of the study,
these should be properly substituted by nan.

#### 1.3.1 - Nan based on maximum values
The study encodes answers in a numeric scale, using high, out of scale values, to encode missing, refusals or invalid answers.

In this way, by looking at the maximum value for the column, we can infer which
are the missing values used. In particular:
- If MAX in [6666, 7777, 8888, 9999] then these will be the values to be mapped to nan
- If MAX in [666, 777, 888, 999] then these will be the values to be mapped to nan
- If MAX in [66, 77, 88, 99] then these will be the values to be mapped to nan
- If MAX in [6, 7, 8, 9] then these will be the values to be mapped to nan

While this method might lead to some mistake, it provides a fast way to start playing around with the data. The nan will be validated again for the final chosen variables.

In [114]:
df_summary["MAX"].value_counts().sort_index(ascending=False)

9999.0     4
8888.0    13
6666.0     3
999.0      1
888.0     15
99.0       3
88.0      36
77.0       4
66.0       4
10.0       1
9.0        2
8.0       47
7.0        3
6.0        7
3.0        1
2.0        2
1.0       20
Name: MAX, dtype: int64

We apply the replacement:

In [115]:
na_batches = [
    (6666, 7777, 8888, 9999),
    (666, 777, 888, 999),
    (66, 77, 88, 99),
    (6, 7, 8, 9),
]

for col in df_summary[df_summary["IS_NUMERIC"] == 1].index:
    for batch in na_batches:
        if df[col].max() in batch:
            df[col] = df[col].apply(lambda x: np.nan if x in batch else x)
            break

In [116]:
df_summary = get_summary_table(df, target_col="happy")
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
nwspol,1,295,91,0.03,0.0,1233.0,173.90,60.0,0.04,0.22
netusoft,1,5,5,0.00,1.0,5.0,3.63,5.0,0.26,0.51
netustm,1,60,1064,0.39,0.0,900.0,165.04,120.0,0.01,0.17
ppltrst,1,11,9,0.00,0.0,10.0,4.79,5.0,0.19,0.18
pplfair,1,11,25,0.01,0.0,10.0,4.83,5.0,0.20,0.20
...,...,...,...,...,...,...,...,...,...,...
uempla,1,2,0,0.00,0.0,1.0,0.08,0.0,-0.10,0.92
uempli,1,2,0,0.00,0.0,1.0,0.03,0.0,-0.02,0.97
dsbld,1,2,0,0.00,0.0,1.0,0.01,0.0,-0.02,0.99
rtrd,1,2,0,0.00,0.0,1.0,0.26,0.0,-0.16,0.74


#### 1.3.2 - Non numeric variables
For non numeric variables missing values are still encoded as 777, 888, 6666, 7777, 8888 or 9999.

In [117]:
filt = df_summary["IS_NUMERIC"]==0
non_numeric_cols = df_summary[filt].index
df_summary[filt]

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
ctzshipd,0,22,0,0.0,NaN,NaN,NaN,NaN,NaN,0.94
cntbrthd,0,27,0,0.0,NaN,NaN,NaN,NaN,NaN,0.90
lnghom1,0,23,0,0.0,NaN,NaN,NaN,NaN,NaN,0.87
lnghom2,0,20,0,0.0,NaN,NaN,NaN,NaN,NaN,0.80
fbrncntc,0,28,0,0.0,NaN,NaN,NaN,NaN,NaN,0.90
mbrncntc,0,28,0,0.0,NaN,NaN,NaN,NaN,NaN,0.90


In [118]:
na_vals = [777, 888, 6666, 7777, 8888, 9999]
for col in non_numeric_cols:
    df[col] = df[col].apply(lambda x: np.nan if x in na_vals else x)

In [119]:
df_summary = get_summary_table(df, target_col="happy")

#### 1.3.3 - Other special cases
For the variables "edulvlb" and "edlveit", 'Other' is mapped as 5555. This throws off the scale. We decide to replace it with np.nan as we are not really able to evaluate it.

In [120]:
for col in ["edulvlb", "edlveit"]:
    df[col] = df[col].replace(5555, np.nan)

### 1.4 - Drop variables with too many missing values
We drop the columns with at least 40% missing values.

In [121]:
summary_filt = df_summary[df_summary["NA_REL"] >= 0.4]
too_many_nan_cols = summary_filt.index
summary_filt

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
prtvtcit,1,14,1463,0.53,1.0,14.0,6.27,7.0,0.01,0.39
prtcldit,1,13,1691,0.62,1.0,14.0,6.54,7.0,-0.01,0.36
prtdgcl,1,4,1699,0.62,1.0,4.0,2.00,2.0,-0.00,0.71
rlgblge,1,2,2177,0.79,1.0,2.0,1.76,2.0,0.02,0.76
rlgdnme,1,4,2612,0.95,1.0,7.0,1.11,1.0,-0.02,0.98
ctzshipd,0,19,2612,0.95,NaN,NaN,NaN,NaN,NaN,0.28
cntbrthd,0,24,2531,0.92,NaN,NaN,NaN,NaN,NaN,0.24
livecnta,1,50,2511,0.91,1940.0,2018.0,1998.74,2002.0,0.11,0.07
fbrncntc,0,25,2513,0.92,NaN,NaN,NaN,NaN,NaN,0.22
mbrncntc,0,25,2514,0.92,NaN,NaN,NaN,NaN,NaN,0.22


In [122]:
df.drop(columns=too_many_nan_cols, inplace=True)

In [123]:
df_summary = get_summary_table(df, target_col="happy")
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
nwspol,1,295,91,0.03,0.0,1233.0,173.90,60.0,0.04,0.22
netusoft,1,5,5,0.00,1.0,5.0,3.63,5.0,0.26,0.51
netustm,1,60,1064,0.39,0.0,900.0,165.04,120.0,0.01,0.17
ppltrst,1,11,9,0.00,0.0,10.0,4.79,5.0,0.19,0.18
pplfair,1,11,25,0.01,0.0,10.0,4.83,5.0,0.20,0.20
...,...,...,...,...,...,...,...,...,...,...
uempla,1,2,0,0.00,0.0,1.0,0.08,0.0,-0.10,0.92
uempli,1,2,0,0.00,0.0,1.0,0.03,0.0,-0.02,0.97
dsbld,1,2,0,0.00,0.0,1.0,0.01,0.0,-0.02,0.99
rtrd,1,2,0,0.00,0.0,1.0,0.26,0.0,-0.16,0.74


### 1.5 - Drop variables where less than 2% of the observations have a value different from the most frequent one
There are some columns attaining a single value for the almost totality of the observations. While they might be useful to diagnose specific subsets of the population, they would hardly help us constructing our first models.

In [124]:
summary_filt = df_summary[df_summary["SHARE_MOST_FREQ"] > 0.97]
summary_filt

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ
wrkprty,1,2,16,0.01,1.0,2.0,1.99,2.0,-0.02,0.99
dscrrce,1,2,0,0.00,0.0,1.0,0.01,0.0,0.00,0.99
dscrntn,1,2,0,0.00,0.0,1.0,0.02,0.0,-0.04,0.98
dscrrlg,1,2,0,0.00,0.0,1.0,0.01,0.0,-0.05,0.99
dscrlng,1,2,0,0.00,0.0,1.0,0.00,0.0,-0.01,1.00
dscretn,1,2,0,0.00,0.0,1.0,0.01,0.0,-0.05,0.99
dscrage,1,2,0,0.00,0.0,1.0,0.00,0.0,-0.05,1.00
dscrgnd,1,2,0,0.00,0.0,1.0,0.00,0.0,-0.02,1.00
dscrsex,1,2,0,0.00,0.0,1.0,0.00,0.0,0.02,1.00
dscrdsb,1,2,0,0.00,0.0,1.0,0.00,0.0,-0.03,1.00


We drop these columns

In [125]:
df.drop(columns=summary_filt.index, inplace=True)

### 1.6 - Drop variables with little to no correlation with target variable
We proceed to drop all the variables having an absolute correlation less than
or equal to 2% with the target variable.

We make an exception for "gndr", representing gender, which will be used anyway as a control variable.

In [126]:
df_summary = get_summary_table(df, target_col="happy", descr_df=df_descr)

summary_filt = df_summary[df_summary["CORR_TARGET"].abs() <= 0.02]
low_corr_cols = summary_filt.index
summary_filt

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
admdw,1,3,0,0.00,1.0,3.0,1.99,2.0,0.00,0.34,Administration of how democracy works questions,discrete
admge,1,2,0,0.00,1.0,2.0,1.49,1.0,-0.00,0.51,"Administration of split ballot, ask about fema...",discrete
bctprd,1,2,20,0.01,1.0,2.0,1.93,2.0,0.00,0.93,Boycotted certain products last 12 months,discrete
blgetmg,1,2,42,0.02,1.0,2.0,1.96,2.0,-0.00,0.96,Belong to minority ethnic group in country,discrete
brncntr,1,2,4,0.00,1.0,2.0,1.10,1.0,0.00,0.90,Born in country,discrete
crmvct,1,2,23,0.01,1.0,2.0,1.86,2.0,0.00,0.86,Respondent or household member victim of burgl...,discrete
ctzcntr,1,2,11,0.00,1.0,2.0,1.06,1.0,-0.02,0.94,Citizen of country,discrete
evpdemp,1,2,18,0.01,1.0,2.0,1.32,1.0,-0.02,0.68,Paid employment or apprenticeship at least 3 m...,discrete
facntr,1,2,8,0.00,1.0,2.0,1.10,1.0,0.01,0.90,Father born in country,discrete
gndr,1,2,0,0.00,1.0,2.0,1.53,2.0,-0.01,0.53,Gender,discrete


In [127]:
df.drop(columns=low_corr_cols.difference(["gndr"]), inplace=True) ## not dropping the gndr column from the low_corr_cols

In [128]:
df_summary = get_summary_table(df, target_col="happy", descr_df=df_descr)
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
acldnmr,1,5,38,0.01,1.0,5.0,3.42,4.0,0.16,0.44,Approve if person have child with partner not ...,discrete
actrolga,1,5,67,0.02,1.0,5.0,1.91,2.0,0.12,0.39,Able to take active role in political group,discrete
advcyc,1,5,134,0.05,1.0,5.0,2.82,3.0,0.12,0.43,Approve if person gets divorced while children...,discrete
aesfdrk,1,4,50,0.02,1.0,4.0,2.22,2.0,-0.16,0.56,Feeling of safety of walking alone in local ar...,discrete
aftjbyc,1,5,88,0.03,1.0,5.0,3.39,3.0,0.10,0.38,Approve if person has full- time job while chi...,discrete
...,...,...,...,...,...,...,...,...,...,...,...,...
vote,1,3,61,0.02,1.0,3.0,1.34,1.0,-0.04,0.74,Voted last national election,discrete
vteurmmb,1,6,358,0.13,1.0,65.0,6.96,1.0,-0.14,0.70,Would vote for [country] to remain member of E...,discrete
wrkorg,1,2,21,0.01,1.0,2.0,1.93,2.0,-0.08,0.93,Worked in another organisation or association ...,discrete
yrbrn,1,76,21,0.01,1928.0,2003.0,1967.68,1967.0,0.21,0.02,Year of birth,continuous


In [129]:
summary_filt1 = df_summary[df_summary["CORR_TARGET"].abs() <= 0.05]
low_corr_cols = summary_filt1.index
summary_filt1

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
ageadlt,1,33,105,0.04,0.0,80.0,19.65,20.0,0.03,0.19,Age become adults. SPLIT BALLOT,continuous
agemage,1,36,77,0.03,0.0,75.0,49.15,50.0,0.05,0.34,Age reach middle age. SPLIT BALLOT,continuous
ageoage,1,37,58,0.02,0.0,99.0,67.35,70.0,0.05,0.32,Age reach old age. SPLIT BALLOT,continuous
bthcld,1,2,25,0.01,1.0,2.0,1.39,1.0,0.04,0.61,Ever given birth to/ fathered a child,discrete
contplt,1,2,21,0.01,1.0,2.0,1.90,2.0,-0.03,0.90,Contacted politician or government official la...,discrete
domicil,1,5,9,0.00,1.0,5.0,3.20,3.0,0.04,0.43,"Domicile, respondent's description",discrete
dscrnap,1,2,0,0.00,0.0,1.0,0.96,1.0,0.05,0.96,Discrimination of respondent's group: not appl...,discrete
evmar,1,2,20,0.01,1.0,2.0,1.37,1.0,0.03,0.63,Are or ever been married,discrete
gincdif,1,5,43,0.02,1.0,5.0,1.85,2.0,0.04,0.53,Government should reduce differences in income...,discrete
gndr,1,2,0,0.00,1.0,2.0,1.53,2.0,-0.01,0.53,Gender,discrete


In [130]:
summary_filt1.sort_values("CORR_TARGET", axis=0, ascending=True, inplace=False)

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
iaglptn,1,32,118,0.04,0.0,111.0,21.75,25.0,-0.05,0.36,"Start living with partner not married to, idea...",continuous
sgnptit,1,2,20,0.01,1.0,2.0,1.86,2.0,-0.05,0.86,Signed petition last 12 months,discrete
pstplonl,1,2,33,0.01,1.0,2.0,1.86,2.0,-0.05,0.86,Posted or shared anything about politics onlin...,discrete
pbldmn,1,2,24,0.01,1.0,2.0,1.92,2.0,-0.05,0.92,Taken part in lawful public demonstration last...,discrete
tygrtr,1,42,290,0.11,0.0,222.0,50.42,50.0,-0.05,0.30,"Retire permanently, age too young. SPLIT BALLOT",continuous
gndr2,1,2,503,0.18,1.0,2.0,1.41,1.0,-0.05,0.59,Gender of second person in household,discrete
vote,1,3,61,0.02,1.0,3.0,1.34,1.0,-0.04,0.74,Voted last national election,discrete
contplt,1,2,21,0.01,1.0,2.0,1.90,2.0,-0.03,0.90,Contacted politician or government official la...,discrete
iagrtr,1,34,142,0.05,0.0,222.0,54.30,60.0,-0.03,0.32,"Retire permanently, ideal age. SPLIT BALLOT",continuous
gndr,1,2,0,0.00,1.0,2.0,1.53,2.0,-0.01,0.53,Gender,discrete


### |3|% = {"ageadlt","contplt","iagpnt","iagrtr","lvgptnea",""}, |4|% = {"nwspol","pbldmn","vote","gincdif"}, |5|% = {"agemage","ageoage","sgnptit","dscrnap","iaglptn","pstplonl","pbldmn","tygrtr", "tygpnt"} some variable to consider = {"domicil", "bthcld", "evrmar", "gndr", "gndr2"} 

In [131]:
df.drop(columns=low_corr_cols.difference(["domicil", "bthcld", "evrmar", "gndr", "gndr2"]), inplace=True) ## not dropping the gndr column from the low_corr_cols

#### Removing the columns which is a ballot to know the opinion on approval or disapproval of a thirds person life.
these columns have the word "BALLOT" in their Label column in the description table

In [132]:
df_summary = get_summary_table(df, target_col="happy", descr_df=df_descr)
df_summary[df_summary['LABEL'].str.contains("BALLOT")]

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
acldnmr,1,5,38,0.01,1.0,5.0,3.42,4.0,0.16,0.44,Approve if person have child with partner not ...,discrete
advcyc,1,5,134,0.05,1.0,5.0,2.82,3.0,0.12,0.43,Approve if person gets divorced while children...,discrete
aftjbyc,1,5,88,0.03,1.0,5.0,3.39,3.0,0.10,0.38,Approve if person has full- time job while chi...,discrete
alvgptn,1,5,40,0.01,1.0,5.0,3.47,4.0,0.17,0.44,Approve if person lives with partner not marri...,discrete
anvcld,1,5,66,0.02,1.0,5.0,3.01,3.0,0.14,0.47,Approve if person chooses never to have childr...,discrete
tygledu,1,27,250,0.09,0.0,60.0,15.33,17.0,0.06,0.26,"Leave full-time education, age too young. SPLI...",continuous
tyglvp,1,25,242,0.09,0.0,111.0,24.57,20.0,-0.12,0.29,"Start living with partner not married to, age ...",continuous


In [133]:
ballot_split_col = df_summary[df_summary['LABEL'].str.contains("BALLOT")].index
ballot_split_col

Index(['acldnmr', 'advcyc', 'aftjbyc', 'alvgptn', 'anvcld', 'tygledu',
       'tyglvp'],
      dtype='object')

#### not relavant according to me = {"acldnmr", "advcyc", "aftjbyc", "anvcld", "tygledu", "tyglvp"}

#### relavant according to me = {"alvgptn"}`

In [134]:
df.drop(columns=ballot_split_col.difference(["alvgptn"]), inplace=True) ## not dropping the gndr column from the low_corr_cols

In [135]:
corr_mat = df.corr(method="pearson")

In [142]:
df["happy"]

0       8.0
1       5.0
2       8.0
3       8.0
4       7.0
       ... 
2740    9.0
2741    7.0
2742    8.0
2743    9.0
2744    5.0
Name: happy, Length: 2745, dtype: float64

In [154]:
mat = df.copy()
mat

,netusoft,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,...,chldhhe,domicil,edulvlb,eisced,edlveit,eduyrs,pdwrk,edctn,uempla,rtrd
0,4.0,2.0,3.0,5.0,3.0,1.0,2.0,1.0,3.0,5.0,...,NaN,4.0,313.0,4.0,9.0,13.0,0,0,0,0
1,5.0,5.0,5.0,6.0,3.0,4.0,1.0,3.0,1.0,NaN,...,2.0,4.0,213.0,2.0,4.0,10.0,0,0,0,0
2,5.0,4.0,4.0,5.0,2.0,2.0,2.0,2.0,2.0,6.0,...,2.0,3.0,620.0,6.0,12.0,15.0,1,0,0,0
3,5.0,6.0,6.0,5.0,2.0,2.0,2.0,2.0,3.0,7.0,...,2.0,4.0,620.0,6.0,12.0,20.0,1,0,0,0
4,5.0,2.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,0.0,...,NaN,4.0,213.0,2.0,4.0,8.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740,3.0,2.0,7.0,7.0,4.0,2.0,1.0,2.0,2.0,0.0,...,2.0,3.0,113.0,1.0,2.0,5.0,0,0,0,1
2741,1.0,4.0,5.0,5.0,4.0,1.0,1.0,1.0,1.0,4.0,...,1.0,4.0,213.0,2.0,4.0,13.0,0,0,0,1
2742,5.0,5.0,5.0,4.0,2.0,3.0,3.0,3.0,3.0,4.0,...,NaN,3.0,321.0,3.0,6.0,13.0,0,0,0,1
2743,5.0,2.0,5.0,5.0,3.0,2.0,2.0,3.0,3.0,6.0,...,2.0,3.0,213.0,2.0,4.0,12.0,0,1,0,0


In [156]:
#mat.drop(columns= ["happy"],axis = 1,inplace = True)
mat.drop(columns= "happy", inplace = True)

In [148]:
#mat[[mat.abs() == 1]] = 0
mat.iloc[0,1]

2.0

In [77]:
mat[mat.abs() > 0.85]

,netusoft,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,...,chldhhe,domicil,edulvlb,eisced,edlveit,eduyrs,pdwrk,edctn,uempla,rtrd
netusoft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ppltrst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pplfair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pplhlp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
polintr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eduyrs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.860602,NaN,NaN,NaN,NaN,NaN
pdwrk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
edctn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uempla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    #x = x.drop(columns=drops)

    return drops

In [163]:
multi_col_clmns = remove_collinear_features(mat,0.85)
multi_col_clmns

trstprt | trstplt | 0.89
lvptnyr | pdempyr | 0.9
maryr | pdempyr | 0.88
maryr | lvptnyr | 0.99
agea | pdempyr | 0.93
agea | lvptnyr | 0.95
agea | maryr | 0.94
yrbrn | pdempyr | 0.93
yrbrn | lvptnyr | 0.95
yrbrn | maryr | 0.94
yrbrn | agea | 1.0
eisced | edulvlb | 0.96
edlveit | edulvlb | 0.96
edlveit | eisced | 0.99
eduyrs | edlveit | 0.86


['edlveit', 'agea', 'yrbrn', 'maryr', 'eduyrs', 'lvptnyr', 'trstprt', 'eisced']

In [203]:
df_summary = get_summary_table(df, target_col="happy", descr_df=df_descr)
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
actrolga,1,5,67,0.02,1.0,5.0,1.91,2.0,0.12,0.39,Able to take active role in political group,discrete
aesfdrk,1,4,50,0.02,1.0,4.0,2.22,2.0,-0.16,0.56,Feeling of safety of walking alone in local ar...,discrete
agea,1,75,21,0.01,16.0,90.0,51.28,52.0,-0.21,0.02,"Age of respondent, calculated",continuous
alvgptn,1,5,40,0.01,1.0,5.0,3.47,4.0,0.17,0.44,Approve if person lives with partner not marri...,discrete
atchctr,1,11,15,0.01,0.0,10.0,8.06,8.0,0.22,0.29,How emotionally attached to [country],discrete
...,...,...,...,...,...,...,...,...,...,...,...,...
uempla,1,2,0,0.00,0.0,1.0,0.08,0.0,-0.10,0.92,"Doing last 7 days: unemployed, actively lookin...",discrete
vteurmmb,1,6,358,0.13,1.0,65.0,6.96,1.0,-0.14,0.70,Would vote for [country] to remain member of E...,discrete
wrkorg,1,2,21,0.01,1.0,2.0,1.93,2.0,-0.08,0.93,Worked in another organisation or association ...,discrete
yrbrn,1,76,21,0.01,1928.0,2003.0,1967.68,1967.0,0.21,0.02,Year of birth,continuous


In [202]:
for i in ['edlveit', 'agea', 'yrbrn', 'maryr', 'eduyrs', 'lvptnyr', 'trstprt', 'eisced']:
    df1 = pd.DataFrame(df_summary[df_summary.index.values == i])
    print(df1["LABEL"])


edlveit    Highest level of education, Italy
Name: LABEL, dtype: object
agea    Age of respondent, calculated
Name: LABEL, dtype: object
yrbrn    Year of birth
Name: LABEL, dtype: object
maryr    Year first married
Name: LABEL, dtype: object
eduyrs    Years of full-time education completed
Name: LABEL, dtype: object
lvptnyr    Year first lived with spouse or partner for 3 ...
Name: LABEL, dtype: object
trstprt    Trust in political parties
Name: LABEL, dtype: object
eisced    Highest level of education, ES - ISCED
Name: LABEL, dtype: object


In [205]:
multi_col_clmns = ['edlveit', 'agea', 'eduyrs', 'lvptnyr', 'trstprt', 'eisced','pdempyr']
multi_col_clmns

['edlveit', 'agea', 'eduyrs', 'lvptnyr', 'trstprt', 'eisced', 'pdempyr']

By manually analyzing these variables, we came down to the above list variables which can be removed from the df

In [206]:
df.drop(columns=multi_col_clmns, inplace=True) 
df

,netusoft,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,...,yrbrn2,dvrcdeva,maritalb,chldhhe,domicil,edulvlb,pdwrk,edctn,uempla,rtrd
0,4.0,2.0,3.0,5.0,3.0,1.0,2.0,1.0,3.0,5.0,...,1981.0,2.0,1.0,NaN,4.0,313.0,0,0,0,0
1,5.0,5.0,5.0,6.0,3.0,4.0,1.0,3.0,1.0,NaN,...,1966.0,2.0,6.0,2.0,4.0,213.0,0,0,0,0
2,5.0,4.0,4.0,5.0,2.0,2.0,2.0,2.0,2.0,6.0,...,1959.0,2.0,6.0,2.0,3.0,620.0,1,0,0,0
3,5.0,6.0,6.0,5.0,2.0,2.0,2.0,2.0,3.0,7.0,...,1954.0,2.0,6.0,2.0,4.0,620.0,1,0,0,0
4,5.0,2.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,0.0,...,1980.0,2.0,1.0,NaN,4.0,213.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740,3.0,2.0,7.0,7.0,4.0,2.0,1.0,2.0,2.0,0.0,...,NaN,2.0,5.0,2.0,3.0,113.0,0,0,0,1
2741,1.0,4.0,5.0,5.0,4.0,1.0,1.0,1.0,1.0,4.0,...,NaN,2.0,1.0,1.0,4.0,213.0,0,0,0,1
2742,5.0,5.0,5.0,4.0,2.0,3.0,3.0,3.0,3.0,4.0,...,1957.0,2.0,1.0,NaN,3.0,321.0,0,0,0,1
2743,5.0,2.0,5.0,5.0,3.0,2.0,2.0,3.0,3.0,6.0,...,1965.0,2.0,6.0,2.0,3.0,213.0,0,1,0,0


In [208]:
df_summary = get_summary_table(df, target_col="happy", descr_df=df_descr)
df_summary

,IS_NUMERIC,N_UNIQUE,NA_ABS,NA_REL,MIN,MAX,MEAN,MEDIAN,CORR_TARGET,SHARE_MOST_FREQ,LABEL,TYPE
actrolga,1,5,67,0.02,1.0,5.0,1.91,2.0,0.12,0.39,Able to take active role in political group,discrete
aesfdrk,1,4,50,0.02,1.0,4.0,2.22,2.0,-0.16,0.56,Feeling of safety of walking alone in local ar...,discrete
alvgptn,1,5,40,0.01,1.0,5.0,3.47,4.0,0.17,0.44,Approve if person lives with partner not marri...,discrete
atchctr,1,11,15,0.01,0.0,10.0,8.06,8.0,0.22,0.29,How emotionally attached to [country],discrete
atcherp,1,11,55,0.02,0.0,10.0,5.66,6.0,0.27,0.19,How emotionally attached to Europe,discrete
...,...,...,...,...,...,...,...,...,...,...,...,...
uempla,1,2,0,0.00,0.0,1.0,0.08,0.0,-0.10,0.92,"Doing last 7 days: unemployed, actively lookin...",discrete
vteurmmb,1,6,358,0.13,1.0,65.0,6.96,1.0,-0.14,0.70,Would vote for [country] to remain member of E...,discrete
wrkorg,1,2,21,0.01,1.0,2.0,1.93,2.0,-0.08,0.93,Worked in another organisation or association ...,discrete
yrbrn,1,76,21,0.01,1928.0,2003.0,1967.68,1967.0,0.21,0.02,Year of birth,continuous


In [209]:
df_summary.to_excel("manual_var_sel.xlsx")

In [215]:
df[["imbgeco", "imdfetn", "impcntr","imsmetn","imueclt","imwbcnt"]].corr(method = "pearson")

,imbgeco,imdfetn,impcntr,imsmetn,imueclt,imwbcnt
imbgeco,1.000000,-0.624092,-0.600034,-0.593944,0.778487,0.738672
imdfetn,-0.624092,1.000000,0.824184,0.824261,-0.626388,-0.563450
impcntr,-0.600034,0.824184,1.000000,0.716150,-0.597612,-0.553877
imsmetn,-0.593944,0.824261,0.716150,1.000000,-0.579414,-0.525670
imueclt,0.778487,-0.626388,-0.597612,-0.579414,1.000000,0.747096
imwbcnt,0.738672,-0.563450,-0.553877,-0.525670,0.747096,1.000000
